In [45]:
from random import *
import hashlib

def puissance (a,e,n):
  p = 1
  while e>0:
    if (e % 2 != 0):
      p=(p*a)%n
    a=(a*a)%n
    e=e//2
  return p

In [80]:
# Test puissance
def test_puissance():
    assert(puissance(4,2,5)==1)
test_puissance()

In [75]:
def test_premier_Fermat(n): # test de Fermat
   if ( (puissance(2,n-1,n)==1) and
      (puissance(3,n-1,n)==1) and
      (puissance(5,n-1,n)==1) and
      (puissance(7,n-1,n)==1) and
      (puissance(11,n-1,n)==1) and
      (puissance(13,n-1,n)==1) ):
      return True; # probablement premier (garantie si n<2^15)
   return False

True

In [76]:
# Test premier de Fermat
def test_Fermat():
    assert(test_premier_Fermat(22)==False)
    assert(test_premier_Fermat(647)==True)
test_Fermat()

In [91]:
def temoin_Miller(n, a):
    s = 0
    d = n-1
    while d % 2 == 0:
       d = d//2
    x = puissance(a, d, n)
    if(x == 1 or x == n-1):
        return False
    for i in range(s-1):
        x = puissance(x, 2, n)
        if(x == n-1):
            return False
    return True

In [92]:
# Test témoin de Miller 
def test_temoin_Miller():
    assert(temoin_Miller(3,9)==True)
    assert(temoin_Miller(4,9)==False)
test_temoin_Miller()

In [98]:
# Le GMP recommande k=25 pour avoir une proba d'erreur < 2**-50
def test_premier_Miller_Rabin(n, k):
    if n == 2 or n == 3:
       return True
    if n % 2 == 0:
       return False
    for i in range(k):
        a = randint(2, n-2)
        if(temoin_Miller(n, a)):
            return False
    return True

In [99]:
# Test de primalité de Miller Rabin
def test_premier_Miller():
    assert(test_premier_Miller_Rabin(3,1)==True)
    assert(test_premier_Miller_Rabin(42,5)==False)
test_premier_Miller()

In [5]:
def pgcd(u, v):
  t = 0
  while (v):
    t = u
    u = v
    v = t % v
  if(u<0):
    return -u
  else:
    return u

In [102]:
# Test PGCD
def test_pgcd():
    assert(pgcd(5,4)==1)
    assert(pgcd(222,448)==2)
test_pgcd()

In [113]:
def bezout(a, b):

  # On sauvegarde les valeurs de a et b.
   a0 = a
   b0 = b

  # On laisse invariant p*a0 + q*b0 = a et  r*a0 + s*b0 = b.
   p = 1
   q = 0
   r = 0
   s = 1
   c = 1
   quotient = 1
   nouveau_r = 1
   nouveau_s = 1

  # La boucle principale.
   while (b != 0):
    c = a % b
    quotient = a//b
    a = b
    b = c
    nouveau_r = p - (quotient * r)
    nouveau_s = q - (quotient * s)
    p = r
    q = s
    r = nouveau_r
    s = nouveau_s
   if p < 0:
     p = p+b0
   return p  # on n'a besoin que de p #si p est négatif retourne p+phi

0

In [115]:
# Test théorème de Bezout
def test_Bezout():
    assert(bezout(50,47)==16)
    assert(bezout(15,5)==0)
    assert(bezout(445,457)==38)
test_Bezout()

In [7]:
#Vérifier que le message est plus petit que la clé
def generateurClé(c,b):
  p=randint(c,b)
  q=randint(c,b)
  e=randint(c,b)
  d=randint(c,b)

  while test_premier_Miller_Rabin(p,25)==False:
    p = randint(c,b)
  while test_premier_Miller_Rabin(q,25)==False:
    q = randint(c,b)

  n=p*q
  
  phi = (p - 1)*(q - 1)
 
  while 1>=e>=phi or pgcd(e,phi)!=1 :
      e = randint(c,b)
  d= int(bezout(int(e),int(phi)))
  return e,d,n 

In [8]:
def chiffrer(message,clé,n):
    return puissance(message,clé,n)

86771824590129366

In [9]:
def dechiffrer(messageChiffre,clé,n):
    return puissance(messageChiffre,clé,n)

198122477765

In [10]:
def str_to_int(m):
    s=0
    b=1
    for i in range(len(m)):
        s=s+ord(m[i])*b
        b=b*256
    return s

In [11]:
def int_to_str(c):
    s =""
    q,r= divmod(c,256)
    s=s+str(chr(r))
    while q != 0:
        q,r = divmod(q,256)
        s =s +str(chr(r))
    return s 

In [12]:
#Probleme de taille de l'empreinte
def calculEmpreinte(message): 
    m=hashlib.sha256(repr(message).encode())
    return m.hexdigest()

3f618
3f618
241343817267


In [ ]:
def decoupe(variable,taille,liste,strVariable):
    decoupeIndex = 0
    for i in range(1,len(variable)):
        if(variable[i]!='0' and i%taille==0):
            liste.append(int(variable[decoupeIndex:i]))
            strVariable+=str(int(variable[decoupeIndex:i]))   
            decoupeIndex=i
        if(i==len(variable)-1):
            liste.append(int(variable[decoupeIndex:len(variable)]))
            strVariable+=str(int(variable[decoupeIndex:len(variable)]))

In [ ]:
def chiffrerEnsemble(liste, cle):
       for i in range(len(liste)):
            liste[i]=chiffrer(liste[i],cle[0],cle[1])

In [13]:
def envoieAuthentifie(message,clePrivee,clePublique):
    #Verifie si le message est un string, si oui le convertit en int
    strEmpreinte=""
    strEmpreinteC=""
    messageEmpreinte=[]
    messageParti=[]
    strMessage=""
    if(isinstance(message,str)):
        message=str_to_int(message)
    
    #Calcule de l'empreinte du message
    empreinte =str(str_to_int(calculEmpreinte(message)))
    tailleMessage=int(len(str(message))/(len(str(message))/2))
    tailleEmpreinte=int(len(empreinte)/(len(empreinte)/2))
    
    #Découpe de l'empreinte en 10 pour pouvoir mieux la chiffrer   
    decoupe(empreinte,tailleEmpreinte,messageEmpreinte,strEmpreinte) 
    print("Empreinte: "+empreinte)
    
    #On decoupe le message en plusieurs morceaux
    decoupe(str(message),tailleMessage,messageParti,strMessage)
    #Chiffrage de l'empreinte avce la clé privée de l'envoyeur
    for i in range(0,len(messageEmpreinte)):
        messageEmpreinte[i]=chiffrer(messageEmpreinte[i],clePrivee[0],clePrivee[1])
        strEmpreinteC+=str(messageEmpreinte[i])
    print("Empreinte Chiffré: "+strEmpreinteC)
    
    #Concaténation du message et de l'empreinte chiffrée
    print("Message + empreinte: "+strMessage+strEmpreinteC)
    
    #On chiffre l'ensemble des empreintes et des messages
    chiffrerEnsemble(messageEmpreinte,clePublique)
    chiffrerEnsemble(messageParti,clePublique)   
        
    #Envoie du message finale chiffrée avec la clé publique du receveur
    return messageParti,messageEmpreinte

In [14]:
def verif_signature(messageParti,messageEmpreinte,clePrivee,clePublique):
    messageEmpreinteDecryptage = messageEmpreinte
    messagePartiDecryptage= messageParti
    empreinte1=""
    empreinteDecrypt=""
    message=""
    
    #Dechiffrage de l'empreinte
    for i in range(len(messageEmpreinteDecryptage)):
      messageEmpreinteDecryptage[i] = dechiffrer(messageEmpreinteDecryptage[i],clePrivee[0],clePrivee[1])
      #print("Empreinte bout déchiffré: "+str(messageListDecryptage[i]))
      empreinte1 += str(messageEmpreinteDecryptage[i])
    
    #Dechiffrage du message
    for i in range(len(messagePartiDecryptage)):
      messagePartiDecryptage[i]=dechiffrer(messagePartiDecryptage[i],clePrivee[0],clePrivee[1])
      message+=str(messagePartiDecryptage[i])
    
    print("Message déchiffré: "+message+empreinte1)
    print("Message : "+message)
    print("Empreinte du message chiffrée: "+empreinte1)
    
    
    #Déchiffrage de l'empreinte avec la clé publique
    for i in range(len(messageEmpreinteDecryptage)):
      messageEmpreinteDecryptage[i] = str(dechiffrer(messageEmpreinteDecryptage[i],clePublique[0],clePublique[1]))
      empreinteDecrypt+= str(messageEmpreinteDecryptage[i])
    
    #Vérification que l'empreinte déchiffrer correspond à celui du message
    if(str_to_int(calculEmpreinte(int(message))))==(int(empreinteDecrypt)):
       return int(message) #int_to_str() demander si 2 fonctions différente ou 1 seule
    else:
       return print("Le message a été modifier")
    

In [15]:
#Alice
gA=generateurClé(10**38,10**39)
cléPubliqueA= [gA[0],gA[2]]
cléPrivéeA= [gA[1],gA[2]]
print("Clé Publique A: "+str(cléPubliqueA))
print("Clé Privée A: "+str(cléPrivéeA))

Clé Publique A: [352577345169288208510792811083652656631, 261901949925735237685632166901063937475412219242419761792740189428368637701597]
Clé Privée A: [139824956444176089112255046364954928216548257808324594678042049760851141319591, 261901949925735237685632166901063937475412219242419761792740189428368637701597]


In [16]:
#CA
gCA=generateurClé(10**56,10**57) 
cléPubliqueCA= [gCA[0],gCA[2]]
cléPrivéeCA= [gCA[1],gCA[2]]
print("Clé Publique CA: "+str(cléPubliqueCA))
print("Clé Privée CA: "+str(cléPrivéeCA))

Clé Publique CA: [581137101231891268397124325021928434994291435151501998109047, 690403777389423963796256044683197758729476646521574254707466670025746136579082282115717082808635754722600895430322589493]
Clé Privée CA: [262850253724056267428164663040378044058681427259983547113991940238888165452373328507583054048025512985933367392227183567, 690403777389423963796256044683197758729476646521574254707466670025746136579082282115717082808635754722600895430322589493]


In [17]:
#Envoie le message au CA
messageA=envoieAuthentifie(int(cléPubliqueA[0]),cléPrivéeA,cléPubliqueCA)

Empreinte: 430456256819
EmpreinteChiffré: 26442151457466804847542999629550812705760713289290816475156642725964758495739
Message + empreinte: 35257734516928820851079281108365265663126442151457466804847542999629550812705760713289290816475156642725964758495739
Message de Alice au CA: (617260313391719916393514583256305030740878284037558407998845923116252320862854462356462201336139534525486917232566858769, 77)


In [18]:
#Vérifie la signature 
if(int(verif_signature(messageA[0],messageA[1],cléPrivéeCA,cléPubliqueA))==cléPubliqueA[0]):
    CAClePubliqeA=chiffrer(cléPubliqueA[0],cléPrivéeCA[0],cléPrivéeCA[1])
    print("Certificat de Alice: "+str(CAClePubliqeA))
else:
    print("Le message a été modifié")

message déchiffré: 35257734516928820851079281108365265663126442151457466804847542999629550812705760713289290816475156642725964758495739
Message: 352577345169288208510792811083652656631
Empreinte du message chiffrée: 26442151457466804847542999629550812705760713289290816475156642725964758495739
ed 430456256819
ec 430456256819
Certificat de Alice: 120301141719698751018647827495511579342027246865268664842065822669712777833230244915813050478412753138182361004698295680


In [ ]:
#Bob vérifie le certificat d'alice
print("Vérification clé publique Alice:")
verifCA=str(dechiffrer(CAClePubliqeA,cléPubliqueCA[0],cléPubliqueCA[1]))
if(verifCA==cléPubliqueA[0]):
    print(""+str(dechiffrer(CAClePubliqeA,cléPubliqueCA[0],cléPubliqueCA[1])))
else:
    print("Le message a été modifié")

In [20]:
#BOB
gB=generateurClé(10**19,10**20)
cléPubliqueB= [gB[0],gB[2]]
cléPrivéeB= [gB[1],gB[2]]
print("Clé Publique B: "+str(cléPubliqueB))
print("Clé Privée B: "+str(cléPrivéeB))

messagez=1435263726352725355365241625326643625165635625635624365265
messageCd=envoieAuthentifie(messagez,cléPrivéeB,cléPubliqueA)

print("Le message d'origine est "+str(messagez))

messageDk=verif_signature(messageCd[0],messageCd[1],cléPrivéeA,cléPubliqueB)

print("Le message décrypté est "+str(messageDk))

Clé Publique B: [718642887184481845, 235850876786633722728899964724486901]
Clé Privée B: [218024657165777089803734356700114413, 235850876786633722728899964724486901]
Empreinte: 207117889891
EmpreinteChiffré: 24161234329075020814480645117859057
Message + empreinte: 2384774824161234329075020814480645117859057
Le message crypté est
(48755692972639776795248872242861326107743431112417160610146153831670493739909, 35)
message déchiffré: 2384774824161234329075020814480645117859057
Message: 23847748
Empreinte du message chiffrée: 24161234329075020814480645117859057
ed 207117889891
ec 207117889891
Le message d'origine est
23847748
Le message décrypté est
23847748
